In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import random
import unicodedata


# Preparamos variables
url_list = list()
productos = list()

#
# Comenzaremos descargando el mapa de la pagina desde  https://www.expert.es/sitemap_index.xml
url = "https://www.expert.es/sitemap_index.xml"

# Para evitar los bloqueos simularemos ser un navegador e incluso cambiaremos de user_agent para
# evitar los bloqueos en las paginas de los productos.
# Informacion de los user agent obtenida de https://developers.whatismybrowser.com/useragents/explore/
# User-Agent: Mozilla/<version> (<system-information>) <platform> (<platform-details>) <extensions>
user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]

print("Iniciamos web scraping...")

# Generamos la consulta
user_agent = random.choice(user_agent_list)
headers = {'User-Agent': user_agent}
page = requests.get(url,headers=headers)

if (page.status_code == 200):

    # Esperamos un tiempo para simular el usuario
    time.sleep(1)
    # Extraemos las etiquetas loc de los enlaces a las paginas
    soup = BeautifulSoup(page.content, 'html.parser')
    url_sitemap = soup.findAll("loc")
    #print(url_sitemap)

    # Recorremos cada una de las paginas buscando enlaces a paginas de productos.
    print("...Recorremos las paginas")
    for each_site in url_sitemap:
        
        # Generamos la consulta
        user_agent = random.choice(user_agent_list)
        headers = {'User-Agent': user_agent}
        page = requests.get(each_site.text,headers=headers)
        print ("%s , %i" %(each_site.text, page.status_code))

        if (page.status_code == 200):
            print("...Buscamos productos")
            soup = BeautifulSoup(page.content, 'html.parser')
            #print(soup)
            full_tag = soup.findAll("loc")
            #print(full_tag)
            
            for each_tag in full_tag:
                # Filtramos para quedarnos solo con las del dominio /producto
                if each_tag.text.find('https://www.expert.es/producto/') != -1:
                    print(each_tag.text)
                    url_list.append(each_tag.text)
    
        # Esperamos un tiempo aleatorio entre peticiones del bucle
        time.sleep(1 + random.randrange(5))
    
    # Comprobamos
    print ("...Se han encontrado %i enlaces" %len(url_list))
    #print(url_list)

else:
    print("La pagina solicitada no responde!")
    print("......Respuesta de la pagina : ", page.status_code)

# ------------------------------------------------------------------------------------------
if len(url_list) == 0:
     
    # Como hemos sido "Baneados" cargo un fichero de texto obtenido de las paginas 
    # https://www.expert.es/product-sitemap1.xml
    # https://www.expert.es/product-sitemap1.xml
    # https://www.expert.es/product-sitemap3.xml
    # antes del bloqueo, y continuamos el script como si lo hubieramos obtenido de la web.
    
    print("Leemos el fichero de texto...")
    url_list = list()
    url_file = open('sites.txt')
    for linea in url_file:
        url_list.append(linea.replace("\n",""))
    print ("...Se han encontrado %i enlaces" %len(url_list))
# -------------------------------------------------------------------------------------------

# Recorremos cada una de las paginas del producto obteniendo la descripcion y el precio.    
print("Recorremos los sites...")

f =  open('electrodata.csv', 'w', newline='',encoding="utf-8")
f.write("Descripcion,Precio,Agotado,Categoria\n")

descripcion = ""
precio = ""
agotado = "NO"
categoria = ""

for url in url_list:
    
     # Generamos la consulta
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    page = requests.get(url,headers=headers)
    print ("%s , %i" %(url, page.status_code))
    
    if (page.status_code == 200):
        
        #time.sleep(1)
        
        print("...Buscamos los datos de los productos")
        soup = BeautifulSoup(page.content, 'html.parser')
        #print(soup)
        
        try:
            # Nombre/Titulo del producto
            # Ejemplo:
            # <h1 class="product_title entry-title" itemprop="name">ENCIMERA GAS ELECTROLUX EGT6633NOK</h1>
            #print ("...Obtenemos los nombres")
            tags = soup.find('h1', class_='product_title entry-title')
            descripcion = tags.getText()
            # Forzamos la normalización
            descripcion=unicodedata.normalize("NFKD",descripcion)
            
            # Precio del producto
            # Ejemplo:
            # #<span class="product-price"><span class="woocommerce-Price-amount amount">550<span class="woocommerce-Price-currencySymbol">€</span></span></span>
            #print ("...Obtenemos los precios")
            # Reemplazamos la coma por el punto y eliminamos el caracter €
            tags = soup.find('span', class_='product-price')
            precio = tags.getText()
            precio = precio.replace(",",".")
            precio = precio.replace('€','')
            # Forzamos la normalización
            precio=unicodedata.normalize("NFKD",precio)
            
            # Categoria del producto
            #print ("...Obtenemos la categoria")
            ##tags=soup.find('div', class_='col-wrapper title-wrapper')
            tags = soup.find('nav', class_= 'like-h1-style woocommerce-breadcrumb')
            categoria = tags.getText()
            # Recortamos la cadena
            categoria = categoria[1:categoria.find("|")]
            # Forzamos la normalización
            categoria=unicodedata.normalize("NFKD",categoria)
            
            
            # Existencia del producto
            # En algunos productos hemos encontrado este flag
            # que queremos registar tambien.
            #print("....Miramos si está agotado")
            tags=None
            agotado = "NO"
            tags=soup.find('div', class_='out-of-stock-flag')
            if (tags):
                agotado="SI"
            # Forzamos la normalización
            agotado=unicodedata.normalize("NFKD",agotado)
            
            print ("%s,%s,%s,%s" %(descripcion, precio, agotado, categoria))
            f.write("%s,%s,%s,%s\n" %(descripcion, precio, agotado, categoria))   
        
        except:
            print("...No se pudo encontrar alguno de los datos.")

    time.sleep(1)
    #time.sleep(1 + random.randrange(5))

f.close()

Iniciamos web scraping...
...Recorremos las paginas
https://www.expert.es/post-sitemap.xml , 200
...Buscamos productos
https://www.expert.es/page-sitemap.xml , 200
...Buscamos productos
https://www.expert.es/product-sitemap1.xml , 200
...Buscamos productos
https://www.expert.es/producto/auriculares-diadema-panasonic-rp-hf100me-a
https://www.expert.es/producto/auriculares-diadema-panasonic-rp-hf100me-k
https://www.expert.es/producto/auriculares-diadema-panasonic-rp-hf100me-w
https://www.expert.es/producto/dvd-reproductor-sony-dvp-sr370
https://www.expert.es/producto/plancha-de-vapor-taurus-geyser-eco-2800
https://www.expert.es/producto/reloj-actividad-deportiva-sunstech-fitlifeprobk
https://www.expert.es/producto/reloj-actividad-deportiva-sunstech-fitlifeprord
https://www.expert.es/producto/altavoz-portatil-spc-internet-4411n-bang-speaker
https://www.expert.es/producto/encimera-vitroceramica-electrica-zanussi-z6123iok
https://www.expert.es/producto/encimera-vitroceramica-electrica-beko-

https://www.expert.es/product-sitemap2.xml , 200
...Buscamos productos
https://www.expert.es/producto/batidora-de-varilla-bosch-msm67170
https://www.expert.es/producto/radio-portatil-sony-icf-p26
https://www.expert.es/producto/radio-portatil-sony-icf-p36
https://www.expert.es/producto/receptor-television-digital-terrestre-sunstech-dtbp700hd2
https://www.expert.es/producto/exprimidor-jata-ex296
https://www.expert.es/producto/plancha-de-asar-jata-gr558
https://www.expert.es/producto/freidora-jata-fr278
https://www.expert.es/producto/plancha-de-vapor-bosch-tda5028020
https://www.expert.es/producto/aspirador-escoba-rowenta-rh9252wo-air-force-all-in-one
https://www.expert.es/producto/horno-electrico-siemens-hb578g0s00
https://www.expert.es/producto/aspirador-trineo-bosch-bgl2ua200
https://www.expert.es/producto/plancha-de-vapor-bosch-tdi902839w
https://www.expert.es/producto/hornillo-gas-vitrokitchen-300bb
https://www.expert.es/producto/hornillo-gas-vitrokitchen-200bb
https://www.expert.es/

https://www.expert.es/product-sitemap3.xml , 200
...Buscamos productos
https://www.expert.es/producto/blu-ray-reproductor-sony-bdp-s1700
https://www.expert.es/producto/cortapelo-braun-hc-5010
https://www.expert.es/producto/ordenador-portatil-primux-iox-all-in-one
https://www.expert.es/producto/cafetera-de-capsulas-delonghi-en-85l-essenza-mini
https://www.expert.es/producto/encimera-vitroceramica-de-induccion-siemens-eh651fdc1e
https://www.expert.es/producto/afeitadora-facial-braun-serie-3-3020s
https://www.expert.es/producto/congelador-vertical-hisense-fv105d4aw2
https://www.expert.es/producto/congelador-vertical-hisense-fv341n4bc2
https://www.expert.es/producto/frigorifico-una-puerta-hisense-rr154d4aw2
https://www.expert.es/producto/frigorifico-dos-puertas-hisense-rt417n4dc1
https://www.expert.es/producto/conjunto-horno-encimera-vitro-beko-bse22120x
https://www.expert.es/producto/campana-telescopica-teka-cnl-6415-negra
https://www.expert.es/producto/campana-telescopica-teka-tl-6310-bl

https://www.expert.es/pa_versinbluetooth-sitemap.xml , 200
...Buscamos productos
...Se han encontrado 1097 enlaces
Recorremos los sites...
https://www.expert.es/producto/auriculares-diadema-panasonic-rp-hf100me-a , 200
...Buscamos los datos de los productos
AURICULARES DIADEMA PANASONIC RP-HF100ME-A,15.90,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/auriculares-diadema-panasonic-rp-hf100me-k , 200
...Buscamos los datos de los productos
AURICULARES DIADEMA PANASONIC RP-HF100ME-K,15.90,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/auriculares-diadema-panasonic-rp-hf100me-w , 200
...Buscamos los datos de los productos
AURICULARES DIADEMA PANASONIC RP-HF100ME-W,15.90,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/dvd-reproductor-sony-dvp-sr370 , 200
...Buscamos los datos de los productos
DVD REPRODUCTOR SONY DVP-SR370,45,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/plancha-de-vapor-taurus-geyser-eco-2800 , 200
...Buscamos los datos de los productos
PLANCHA DE VAPOR

https://www.expert.es/producto/altavoz-portatil-sunstech-muscle-pro-bl , 200
...Buscamos los datos de los productos
ALTAVOZ PORTÁTIL SUNSTECH MUSCLE PRO BL,119,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/radio-portatil-sunstech-rpc5-blis-sl , 200
...Buscamos los datos de los productos
RADIO PORTÁTIL SUNSTECH RPC5 BLIS SL,12.90,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/altavoz-portatil-sunstech-spbtaqua-bl , 200
...Buscamos los datos de los productos
ALTAVOZ PORTÁTIL SUNSTECH SPBTAQUA BL,29.90,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/smartphone-alcatel-a5-led-5085d-plata , 200
...Buscamos los datos de los productos
SMARTPHONE ALCATEL A5 LED (5085D) PLATA,99,NO,INFORMÁTICA Y TELEFONÍA  
https://www.expert.es/producto/auriculares-boton-sony-sbh20-rosa , 200
...Buscamos los datos de los productos
AURICULARES BOTÓN SONY SBH20 ROSA,29.90,NO,IMAGEN Y SONIDO  
https://www.expert.es/producto/dron-parrot-rolling-spider-rojo , 200
...Buscamos los datos de los pr

...

https://www.expert.es/producto/batidora-de-varilla-braun-hm-3100-wh , 200
...Buscamos los datos de los productos
BATIDORA MULTIFUNCIÓN BRAUN HM 3100 WH,55,SI,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/batidora-de-vaso-braun-jb-3060-wh-2018 , 200
...Buscamos los datos de los productos
BATIDORA DE VASO BRAUN JB 3060 WH (2018),75,SI,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/encimera-vitroceramica-electrica-teka-tz-6317 , 200
...Buscamos los datos de los productos
ENCIMERA VITROCERÁMICA ELÉCTRICA TEKA TZ 6317,239,NO,GRAN ELECTRODOMÉSTICO  
https://www.expert.es/producto/almohadilla-electrica-normal-daga-flexyheat-colors , 200
...Buscamos los datos de los productos
ALMOHADILLA ELÉCTRICA NORMAL DAGA FLEXYHEAT COLORS,29.90,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/campana-decorativa-mepamsa-gavia-70-inox , 200
...Buscamos los datos de los productos
CAMPANA DECORATIVA DE PARED MEPAMSA GAVIA 70 INOX,159,NO,GRAN ELECTRODOMÉSTICO  
htt

https://www.expert.es/producto/radiador-termoceramico-orbegozo-rrm-1510 , 200
...Buscamos los datos de los productos
EMISORES TÉRMICOS ORBEGOZO RRM 1510,159,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/radiador-termoceramico-orbegozo-rrm-1310 , 200
...Buscamos los datos de los productos
EMISORES TÉRMICOS ORBEGOZO RRM 1310,139,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/radiador-termoceramico-orbegozo-rrm-1810 , 200
...Buscamos los datos de los productos
EMISORES TÉRMICOS ORBEGOZO RRM 1810,189,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/radiador-termoceramico-orbegozo-rre-1010 , 200
...Buscamos los datos de los productos
EMISORES TÉRMICOS ORBEGOZO RRE 1010,129,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/radiador-termoceramico-orbegozo-rre-810 , 200
...Buscamos los datos de los productos
EMISORES TÉRMICOS ORBEGOZO RRE 810,119,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/calienta-camas-orbegozo-ce-6000 , 200
...Buscamos los datos de los productos
CALI

https://www.expert.es/producto/calefactor-termoventilador-syp-tl-39-h , 200
...Buscamos los datos de los productos
CALEFACTOR TERMOVENTILADOR SYP TL-39 H,32.90,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/calefactor-termoventilador-syp-tl-39-v , 200
...Buscamos los datos de los productos
CALEFACTOR TERMOVENTILADOR SYP TL-39 V,39.90,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/calefactor-termoventilador-syp-tl-39-vm , 200
...Buscamos los datos de los productos
CALEFACTOR TERMOVENTILADOR SYP TL-39 VM,49.90,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/almohadilla-electrica-normal-daga-flexy-heat-e4 , 200
...Buscamos los datos de los productos
ALMOHADILLA ELÉCTRICA NORMAL DAGA FLEXY-HEAT E4,47,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/radiador-de-infrarrojos-orbegozo-bp-0206 , 200
...Buscamos los datos de los productos
RADIADOR DE INFRARROJOS ORBEGOZO BP 0206,20.90,NO,CLIMATIZACIÓN  
https://www.expert.es/producto/emisores-termicos-orbegozo-reh-2060

https://www.expert.es/producto/horno-electrico-teka-hbe-615-me-ss , 200
...Buscamos los datos de los productos
HORNO ELÉCTRICO TEKA HBE 615 ME SS,339,NO,GRAN ELECTRODOMÉSTICO  
https://www.expert.es/producto/cafetera-de-capsulas-krups-kp1301-lumio-blanca , 200
...Buscamos los datos de los productos
CAFETERA DE CÁPSULAS KRUPS KP1301 LUMIO BLANCA,109,SI,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/barbero-braun-bt5240 , 200
...Buscamos los datos de los productos
BARBERO BRAUN BT5240,45.90,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cortapelo-braun-mgk5260 , 200
...Buscamos los datos de los productos
CORTAPELO BRAUN MGK5260,59,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cortapelo-braun-mgk7221 , 200
...Buscamos los datos de los productos
CORTAPELO BRAUN MGK7221,79,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cortapelo-philips-hc5612-15 , 200
...Buscamos los datos de los productos
CORTAPELO PHILIPS HC5612 15,32.90,NO

https://www.expert.es/producto/cafetera-de-goteo-orbegozo-cg-4016 , 200
...Buscamos los datos de los productos
CAFETERA DE GOTEO ORBEGOZO CG 4016,22.90,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cafetera-fuego-orbegozo-kfi-960 , 200
...Buscamos los datos de los productos
CAFETERA FUEGO ORBEGOZO KFI 960,17.90,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cafetera-fuego-orbegozo-kf-300 , 200
...Buscamos los datos de los productos
CAFETERA FUEGO ORBEGOZO KF 300,5.90,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cafetera-fuego-orbegozo-kf-900 , 200
...Buscamos los datos de los productos
CAFETERA FUEGO ORBEGOZO KF 900,9.90,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cafetera-fuego-orbegozo-kf-1200 , 200
...Buscamos los datos de los productos
CAFETERA FUEGO ORBEGOZO KF 1200,11.90,NO,PEQUEÑO ELECTRODOMÉSTICO  
https://www.expert.es/producto/cafetera-fuego-orbegozo-kf-600 , 200
...Buscamos los datos de los productos
CAF

In [6]:
import pandas as pd
df=pd.read_csv("electrodata.csv")
df.head()

,Descripcion,Precio,Agotado,Categoria
0,AURICULARES DIADEMA PANASONIC RP-HF100ME-A,15.9,NO,IMAGEN Y SONIDO
1,AURICULARES DIADEMA PANASONIC RP-HF100ME-K,15.9,NO,IMAGEN Y SONIDO
2,AURICULARES DIADEMA PANASONIC RP-HF100ME-W,15.9,NO,IMAGEN Y SONIDO
3,DVD REPRODUCTOR SONY DVP-SR370,45.0,NO,IMAGEN Y SONIDO
4,PLANCHA DE VAPOR TAURUS GEYSER ECO 2800,49.0,NO,PEQUEÑO ELECTRODOMÉSTICO


In [7]:
df['Categoria'].unique()

array(['IMAGEN Y SONIDO  ', 'PEQUEÑO ELECTRODOMÉSTICO  ',
       'INFORMÁTICA Y TELEFONÍA  ', 'GRAN ELECTRODOMÉSTICO  ',
       'CLIMATIZACIÓN  '], dtype=object)

In [9]:
100 * df['Categoria'].value_counts() / len(df['Categoria'])

PEQUEÑO ELECTRODOMÉSTICO      31.423290
IMAGEN Y SONIDO                 20.887246
GRAN ELECTRODOMÉSTICO          18.391867
INFORMÁTICA Y TELEFONÍA       14.695009
CLIMATIZACIÓN                  14.602588
Name: Categoria, dtype: float64

In [10]:
100 * df['Agotado'].value_counts() / len(df['Agotado'])

NO    93.900185
SI     6.099815
Name: Agotado, dtype: float64

In [11]:
df['Precio'].describe()

count    1082.000000
mean      125.670939
std       165.375629
min         1.099000
25%        27.900000
50%        55.000000
75%       149.000000
max       999.000000
Name: Precio, dtype: float64